In [1]:
# Copyright 2022 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

<img src="http://developer.download.nvidia.com/compute/machine-learning/frameworks/nvidia_logo.png" style="width: 90px; float: right;">

# Multi-GPU training with Tensorflow and Horovod

This notebook is created using the latest stable [merlin-tensorflow](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/merlin/containers/merlin-tensorflow) container.

## Overview

In this notebook we will look at multi-GPU training with Tensorflow and Horovod. [Horovod] is a distributed deep learning framework that aims to make distributed deep learning fast and easy to use.

In this example, we will provide a simple pipeline to train a MatrixFactorization Model in TensorFlow on multiple GPUs (for the example we will use two but this method can be easily extended to use more).

### Learning objectives

- Training on multiple GPUs with Merlind Dataloader and Horovod.

# Downloading and preparing the dataset

We will base our example on the  [MovieLens25M](https://grouplens.org/datasets/movielens/25m/) dataset.

In [2]:
from merlin.core.utils import download_file
from merlin.core.dispatch import get_lib

In [3]:
DATA_PATH = '/workspace'
download_file("http://files.grouplens.org/datasets/movielens/ml-25m.zip", DATA_PATH + "/ml-25m.zip")

downloading ml-25m.zip: 262MB [00:06, 42.7MB/s]                            
unzipping files: 100%|██████████| 8/8 [00:08<00:00,  1.03s/files]


# Training a TensorFlow Keras Model with Merlin dataloader and Horovod

We will be training with two processes on two GPUs. We will implement data parallel training. Each GPU will have an exact copy of our model, however it will train on different subsets of data.

Let's us split our train data into two parquet files.

In [4]:
ratings = get_lib().read_csv(DATA_PATH + '/ml-25m/ratings.csv')

ratings[:int(0.5*ratings.shape[0])].to_parquet(DATA_PATH + '/train_0.parquet')
ratings[int(0.5*ratings.shape[0]):].to_parquet(DATA_PATH + '/train_1.parquet')

Let us now take a closer look at what else we will need to train with Horovod.

### Write the training script to a file

We need to have a `.py` file we will be able to load into each process using `horovodrun`. 

### Set `CUDA visible devices` correctly inside each process

We need to set the visible device in each process to its `rank`. This way process with `rank 0` will use the zeroth GPU, process with `rank 1` will use the first GPU, and so on. This ensures that each worker can access only a single GPU.

Additionally, we will use the `rank` infromation to select the correct parquet file to load per worker (`DATA_PATH + f'/train_{hvd.local_rank()}.parquet'`)

**Important: Individual parquet files require to have the same number of batches. If one worker has more batches than another, the training process will freeze. At one point during the training process, the worker with more batches waits for the gradients from the worker with fewer batches. But the worker with fewer batches finished the training run.**

In [6]:
%%writefile './tf_trainer.py'

from merlin.io import Dataset

import tensorflow as tf
import horovod.tensorflow.keras as hvd
import os
from glob import glob
os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async"

hvd.init()

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
if gpus:
    tf.config.experimental.set_visible_devices(gpus[hvd.local_rank()], 'GPU')

from merlin.loader.tensorflow import Loader

DATA_PATH = '/workspace'

from merlin.core.dispatch import get_lib
dataset = Dataset(glob(DATA_PATH + f'/train_{hvd.local_rank()}.parquet'), part_mem_fraction=0.001)
loader = Loader(dataset, batch_size=1024, global_rank=hvd.rank(), global_size=hvd.size())

batch = next(iter(loader))

label_column = 'rating'

def process_batch(data, _):
    x = {col: data[col] for col in data.keys() if col != label_column}
    y = data[label_column]
    return (x, y)


loader._map_fns = [process_batch]


class MatrixFactorization(tf.keras.Model):
    def __init__(self, n_factors):
        super().__init__()
        self.user_embeddings = tf.keras.layers.Embedding(162541, n_factors)
        self.movie_embeddings = tf.keras.layers.Embedding(209171, n_factors)

    def call(self, batch, training=False):
        user_embs = self.user_embeddings(batch['userId'])
        movie_embs = self.movie_embeddings(batch['movieId'])

        tensor = (tf.squeeze(user_embs) * tf.squeeze(movie_embs))
        return tf.reduce_sum(tensor, 1)
    
model = MatrixFactorization(64)
opt = tf.keras.optimizers.Adam(1e-2  * hvd.size())
opt = hvd.DistributedOptimizer(opt)
model.compile(optimizer=opt, loss=tf.keras.losses.MeanSquaredError(), experimental_run_tf_function=False)

model.fit(loader, epochs=1, callbacks=[hvd.callbacks.BroadcastGlobalVariablesCallback(0)], verbose=1 if hvd.rank() == 0 else 0)

Overwriting ./tf_trainer.py


In [7]:
!horovodrun -np 2 python tf_trainer.py

[1,0]<stderr>:2022-12-06 06:08:06.849308: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
[1,0]<stderr>:To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
[1,1]<stderr>:2022-12-06 06:08:06.849480: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
[1,1]<stderr>:To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
[1,0]<stderr>:2022-12-06 06:08:07.048602: I tensorflow/core/common_runtime/gpu/gpu_process_state.cc:222] Using CUDA malloc Async allocator for GPU: 0
[1,0]<stderr>:2022-12-06 06:08:07.048936: I tensorflow/core/common_runtime/gpu/gpu

In [9]:
!nvidia-smi

Tue Dec  6 06:09:17 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.119.04   Driver Version: 450.119.04   CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:06:00.0 Off |                    0 |
| N/A   34C    P0    47W / 163W |   1456MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:0B:00.0 Off |                    0 |
| N/A   

## Conclusion

We demonstrated how to train a TensorFlow Keras model with the Merlin dataloader on multiple GPUs using Horovod.

# Next Steps

Merlin dataloader is part of NVIDIA Merlin, a open source framework for recommender systems. In this example, we looked only on a specific use-case to accelerate existing training pipelines. We provide more libraries to make recommender system pipelines easier and faster to work with:

* [NVTabular](https://github.com/NVIDIA-Merlin/NVTabular) is a library to accelerate and scale feature engineering
* [Merlin Models](https://github.com/NVIDIA-Merlin/models) is a library with high-quality implementations of popular recommender systems architectures

The libraries are designed to work closely together. We recommend to check out our examples:

* [Getting Started with NVTabular: Process Tabular Data On GPU](https://github.com/NVIDIA-Merlin/NVTabular/blob/main/examples/01-Getting-started.ipynb)
* [Getting Started with Merlin Models: Develop a Model for MovieLens](https://github.com/NVIDIA-Merlin/models/blob/main/examples/01-Getting-started.ipynb)

In the example, [From ETL to Training RecSys models - NVTabular and Merlin Models integrated example](https://github.com/NVIDIA-Merlin/models/blob/main/examples/02-Merlin-Models-and-NVTabular-integration.ipynb), we explain how the close collaboration works.